In [1]:
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

In [2]:
from export_matomo import *

In [3]:
pull_matomo_data(print_messages=True)

starting data pull
	last date: 2022-12-04; offset: 0
	received 1,000 visitor entries
	processed visitor entries; storing 2,128 page view entries
	last date: 2022-12-04; offset: 1,000
	received 1,000 visitor entries
	processed visitor entries; storing 2,069 page view entries
	last date: 2022-12-04; offset: 2,000
	received 1,000 visitor entries
	processed visitor entries; storing 2,360 page view entries
	last date: 2022-12-04; offset: 3,000
	received 1,000 visitor entries
	processed visitor entries; storing 2,420 page view entries
	last date: 2022-12-04; offset: 4,000
	received 1,000 visitor entries
	processed visitor entries; storing 2,171 page view entries
	last date: 2022-12-04; offset: 5,000
	received 1,000 visitor entries
	processed visitor entries; storing 2,097 page view entries
	last date: 2022-12-04; offset: 6,000
	received 1,000 visitor entries
	processed visitor entries; storing 2,235 page view entries
	last date: 2022-12-04; offset: 7,000
	received 1,000 visitor entries
	proc

Exception: 

https://main.matomo.qgiv.com/?module=API&method=Live.getLastVisitsDetails&idSite=1&period=day&date=2022-11-13&format=json&token_auth=20e9d5eeece2562bb882cb1e39f76828&filter_limit=1000&filter_offset=0

# test call

In [5]:
import requests

url = "https://main.matomo.qgiv.com/?module=API&method=Live.getLastVisitsDetails&idSite=1&period=day&date=2022-11-13&format=json&token_auth=20e9d5eeece2562bb882cb1e39f76828&filter_limit=1000&filter_offset=0"
d = requests.get(url, verify=False)

In [9]:
d.text[:100]

'[{"idSite":"1","idVisit":"3870857","visitIp":"173.175.116.193","visitorId":"8b82da709b012762","finge'

# ingest

In [2]:
files = list_files("matomo-records")

matomo_update.2022-12-15-7.csv (0.03MB)
matomo_update.2022-12-16-7.csv (0.28MB)
matomo_update.2022-12-17-7.csv (0.34MB)
matomo_update.2022-12-18-1.csv (0.33MB)
matomo_update.2022-12-18-7.csv (0.30MB)
matomo_update.2022-12-19-7.csv (0.21MB)
matomo_update.2022-12-20-7.csv (0.03MB)
matomo_update.2022-12-21-7.csv (0.25MB)
matomo_update.2022-12-22-7.csv (0.26MB)
matomo_update.2022-12-23-7.csv (0.10MB)
matomo_update.2022-12-24-7.csv (0.09MB)
matomo_update.2022-12-25-7.csv (0.04MB)
matomo_update.2022-12-26-7.csv (0.26MB)
matomo_update.2022-12-27-7.csv (0.06MB)
matomo_update.2022-12-28-7.csv (0.38MB)
--------------------------------------------------
Bucket matomo-records contains 15 files (0.0GB)


In [15]:
for f in files:     
    # load to redshift
    q = '''copy matomo_traffic
        from 's3://matomo-records/{}'
        iam_role 'arn:aws:iam::637885584661:role/AWSRoleForRedshift'
        emptyasnull
        blanksasnull
        fillrecord
        delimiter ','
        ignoreheader 1
        region 'us-east-1';'''.format(f)

    try:
        redshift_query_write(q, schema='production')
        
        # if successful, delete files from bucket
        #delete_s3_file("matomo-records", f)
    except Exception as e:
        print("error ingesting matomo-records/{}".format(f))
        print(e)